In [22]:
from transformers import pipeline, AutoTokenizer


In [23]:
pairs = [
    ("The capital of France is Berlin.", "The capital of France is Paris."),
    ('I am in California', 'I am in United States.'),
    ('I am in United States', 'I am in California.'),
    ("A person on a horse jumps over a broken down airplane.", "A person is outdoors, on a horse."),
    ("A boy is jumping on skateboard in the middle of a red bridge.", "The boy skates down the sidewalk on a red bridge"),
    ("A man with blond-hair, and a brown shirt drinking out of a public water fountain.", "A blond man wearing a brown shirt is reading a book."),
    ("Mark Wahlberg was a fan of Manny.", "Manny was a fan of Mark Wahlberg.")
]

In [24]:
classifier = pipeline(
    "text-classification",
    model='vectara/hallucination_evaluation_model',
    tokenizer=AutoTokenizer.from_pretrained('google/flan-t5-base'),
    trust_remote_code=True
)

You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.
You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.
c:\Users\prana\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
correction_generator = pipeline(
    "text-generation",
    model='google/flan-t5-base',  
    tokenizer=AutoTokenizer.from_pretrained('google/flan-t5-base'),
    trust_remote_code=True
)


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM'

In [26]:
for premise, hypothesis in pairs:
    prompt = f"<pad> Determine if the hypothesis is true given the premise?\n\nPremise: {premise}\n\nHypothesis: {hypothesis}"
    result = classifier(prompt)[0]  # Assuming single result

    if result['label'] != 'consistent':
        print(f"Hallucination detected for: {premise}")
        print(f"Original hypothesis: {hypothesis}")

        correction_prompt = f"Given the following premise, suggest a correction for the hypothesis if it contains inaccuracies.\n\nPremise: {premise}\n\nHypothesis: {hypothesis}\n\nCorrection:"
        correction = correction_generator(correction_prompt, max_length=50)[0]['generated_text']
        
        print(f"Suggested correction: {correction}")
    else:
        print(f"No hallucination detected for: {premise}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Hallucination detected for: The capital of France is Berlin.
Original hypothesis: The capital of France is Paris.
Suggested correction: Given the following premise, suggest a correction for the hypothesis if it contains inaccuracies.

Premise: The capital of France is Berlin.

Hypothesis: The capital of France is Paris.

Correction:
No hallucination detected for: I am in California
Hallucination detected for: I am in United States
Original hypothesis: I am in California.
Suggested correction: Given the following premise, suggest a correction for the hypothesis if it contains inaccuracies.

Premise: I am in United States

Hypothesis: I am in California.

Correction:
No hallucination detected for: A person on a horse jumps over a broken down airplane.
Hallucination detected for: A boy is jumping on skateboard in the middle of a red bridge.
Original hypothesis: The boy skates down the sidewalk on a red bridge
Suggested correction: Given the following premise, suggest a correction for the 